## Preperation: load dependencies

In [1]:
//import $ivy.`com.typesafe.akka::akka-http-spray-json:10.2.4`
//import $ivy.`com.typesafe.slick::slick:3.3.3`
//import $ivy.`com.typesafe.slick::slick-hikaricp:3.3.3`
//import $ivy.`com.github.tminglei::slick-pg:0.19.5`
//import $ivy.`com.github.tminglei::slick-pg_spray-json:0.19.5`
//import $ivy.`org.postgresql:postgresql:42.2.19`
//import $ivy.`ch.qos.logback:logback-classic:1.2.3`

// Load fat JAR
interp.load.cp(os.Path("/home/jovyan/jar/masterthesis-khipu-assembly-0.1.jar"))

In [2]:
// Smile

import $ivy.`com.github.haifengl::smile-scala:2.6.0`

import scala.language.postfixOps
import org.apache.commons.csv.CSVFormat
import smile._
import smile.util._
import smile.math._
import smile.math.MathEx._
import smile.math.kernel._
import smile.math.matrix._
import smile.math.matrix.Matrix._
import smile.math.rbf._
import smile.stat.distribution._
import smile.data._
import smile.data.formula._
import smile.data.measure._
import smile.data.`type`._
import smile.json._
import smile.interpolation._
import smile.validation._
import smile.association._
import smile.base.cart.SplitRule
import smile.base.mlp._
import smile.base.rbf.RBF
import smile.classification._
import smile.regression._
import smile.feature._
import smile.clustering._
import smile.vq._
import smile.manifold._
import smile.mds._
import smile.sequence._
import smile.projection._
import smile.nlp._
import smile.wavelet._
// Plotting
import smile.plot.swing._
import smile.plot.vega._
import smile.plot.show
import smile.plot.Render._

import $ivy.$                                       


import scala.language.postfixOps

import org.apache.commons.csv.CSVFormat

import smile._

import smile.util._

import smile.math._

import smile.math.MathEx._

import smile.math.kernel._

import smile.math.matrix._

import smile.math.matrix.Matrix._

import smile.math.rbf._

import smile.stat.distribution._

import smile.data._

import smile.data.formula._

import smile.data.measure._

import smile.data.`type`._

import smile.json._

import smile.interpolation._

import smile.validation._

import smile.association._

import smile.base.cart.SplitRule

import smile.base.mlp._

import smile.base.rbf.RBF

import smile.classification._

import smile.regression._

import smile.feature._

import smile.clustering._

import smile.vq._

import smile.manifold._

import smile.mds._

import smile.sequence._

import smile.projection._

import smile.nlp._

import smile.wavelet._
// Plotting

import smile.plot.swing._

import smile.plot.vega._

i

## Get data from DB

In [3]:
import scala.concurrent.ExecutionContext.Implicits.global
import scala.concurrent.Future
import scala.concurrent.duration._
import scala.concurrent.{ Await, Future }
import scala.language.postfixOps

import de.tmrdlt.database.workflowlist.WorkflowListDB
import de.tmrdlt.database.action.ActionDB


val workflowListDB = new WorkflowListDB()
val actionDB = new ActionDB()

val workflowLists =  Await.result(workflowListDB.getWorkflowLists, 2 seconds)
val actions =  Await.result(actionDB.getActions, 2 seconds)

11:20:35.192 [scala-interpreter-1] INFO  MyPostgresProfile$ -  >>> binding uuid -> java.util.UUID 
11:20:35.206 [scala-interpreter-1] INFO  MyPostgresProfile$ -  >>> binding text -> java.lang.String 
11:20:35.206 [scala-interpreter-1] INFO  MyPostgresProfile$ -  >>> binding bool -> Boolean 
11:20:35.497 [scala-interpreter-1] INFO  com.zaxxer.hikari.HikariDataSource - workflow.postgres.db - Starting...
11:20:35.581 [scala-interpreter-1] INFO  com.zaxxer.hikari.HikariDataSource - workflow.postgres.db - Start completed.


import scala.concurrent.ExecutionContext.Implicits.global

import scala.concurrent.Future

import scala.concurrent.duration._

import scala.concurrent.{ Await, Future }

import scala.language.postfixOps


import de.tmrdlt.database.workflowlist.WorkflowListDB

import de.tmrdlt.database.action.ActionDB



workflowListDB: WorkflowListDB = de.tmrdlt.database.workflowlist.WorkflowListDB@4594f41c
actionDB: ActionDB = de.tmrdlt.database.action.ActionDB@54c1892a
workflowLists: Seq[de.tmrdlt.database.workflowlist.WorkflowList] = Vector(
  WorkflowList(
    1L,
    "5fad0555e165558a64ad0478",
    "Masterarbeit",
    Some(""),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-03-08T11:11:54.658
  ),
  WorkflowList(
    4L,
    "5fad0555e165558a64ad047b",
    "Fertig",
    None,
    Some(1L),
    2L,
    LIST,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-02-15T09:43:41.157
  ),
  WorkflowList(
    5L,
    "5fc4d469bcaac0

# Trello
## Size of boards and columns over time

In [8]:
import de.tmrdlt.models._
import java.time.LocalDate
import scala.collection.mutable.ListBuffer

val boards = workflowLists
    .filter(_.listType == WorkflowListType.BOARD)
    .filter(_.dataSource == WorkflowListDataSource.Trello)

boards.foreach { board =>
    val data = new ListBuffer[(Int, LocalDate)]
    data += ((0, board.createdAt.toLocalDate))
    
    actions
        .filter(a => a.actionType == "createCard" || a.actionType == "deleteCard")
        .sortBy(_.date)
        .foreach { a => 
            val count = if (a.actionType == "createCard") data.last._1 + 1 else data.last._1 - 1
            data += ((count, a.date.toLocalDate))
        }
    
    val smileData = data.toList.map(d => JsObject("date" -> JsDate(d._2), "count" -> JsInt(d._1)))
    val chart = VegaLite.view()
        .mark(JsObject("type" -> "area", "line" -> true, "point" -> true))
        .width(800)
        .height(200)
        .x(field = "date", `type` = "temporal")
        .y(field = "count", `type` = "quantitative")
        .data(smileData)
        .title(s"Size of the board '${board.title}' over time")
    //show(chart)
    
    val columns = workflowLists
    .filter(_.listType == WorkflowListType.LIST)
    .filter(_.parentId == Some(board.id))
    .sortBy(_.position)

    val columnData = 
    columns.map { column =>
        val data = new ListBuffer[(Int, LocalDate, String, Int)]
        data += ((0, column.createdAt.toLocalDate, column.title, column.position.toInt))
        
        actions
            .filter(a => a.parentApiId == Some(column.apiId) ||
                         a.oldParentApiId == Some(column.apiId) ||
                         a.newParentApiId == Some(column.apiId))
            .sortBy(_.date)
            .foreach { a => 
                val count = 
                    if (a.actionType == "createCard") {
                        data.last._1 + 1
                    } else if (a.actionType == "deleteCard") {
                        data.last._1 - 1  
                    } else {
                        if (column.apiId == a.oldParentApiId.get) {
                            data.last._1 - 1
                        } else {
                            data.last._1 + 1
                        }
                    } 
                data += ((count, a.date.toLocalDate, column.title, column.position.toInt))
            }
        data
    }
    // print(columnData)
    val columnSmileData = columnData.flatten.toList.map(d => JsObject("card_count" -> JsInt(d._1), "date" -> JsDate(d._2), "column_name" -> JsString(d._3), "position" -> JsInt(d._4)))

    val chart2 = VegaLite.facet(columnSmileData)
        .row(field = "column_name", `type` = "nominal")
        .mark(JsObject("type" -> "area", "line" -> true, "point" -> true))
        .width(800)
        .height(150)
        .x(field = "date", `type` = "temporal")
        .y(field = "card_count", `type` = "quantitative")
        .title(s"Size of the columns of board '${board.title}' over time")
    show(chart2)
}

<iframe id="981c81a1-7e52-4de7-ba6f-53d1f6ac25ec" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"column_name":"tasks","date":"2020-11-12","position":0,"card_count":0},{"column_name":"tasks","date":"2020-11-12","position":0,"card_count":1},{"column_name":"tasks","date":"2020-11-12","position":0,"card_count":2},{"column_name":"tasks","date":"2020-11-12","position":0,"card_count":3},{"column_name":"tasks","date":"2020-11-12","position":0,"card_count":4},{"column_name":"tasks","date":"2020-11-30","position":0,"card_count":5},{"column_name":"tasks","date":"2020-11-30","position":0,"card_count":4},{"column_name":"tasks","date":"2020-11-30","position":0,"card_count":3},{"column_name":"tasks","date":"2020-11-30","position":0,"card_count":2},{"column_name":"tasks","date":"2020-11-30","position":0,"card_count":3},{"column_name":"tasks","date":"2020-11-30","position":0,"card_count":4},{"column_name":"tasks","date":"2020-12-01","position":0,"card_count":5},{"column_name":"tasks","date":"2020-12-01","position":0,"card_count":6},{"column_name":"tasks","date":"2020-12-01","position":0,"card_count":5},{"column_name":"tasks","date":"2020-12-01","position":0,"card_count":4},{"column_name":"tasks","date":"2020-12-04","position":0,"card_count":5},{"column_name":"tasks","date":"2020-12-04","position":0,"card_count":4},{"column_name":"tasks","date":"2020-12-04","position":0,"card_count":3},{"column_name":"tasks","date":"2020-12-04","position":0,"card_count":2},{"column_name":"tasks","date":"2020-12-07","position":0,"card_count":1},{"column_name":"tasks","date":"2020-12-07","position":0,"card_count":0},{"column_name":"tasks","date":"2020-12-07","position":0,"card_count":-1},{"column_name":"tasks","date":"2020-12-07","position":0,"card_count":0},{"column_name":"tasks","date":"2020-12-08","position":0,"card_count":1},{"column_name":"tasks","date":"2020-12-08","position":0,"card_count":0},{"column_name":"tasks","date":"2020-12-14","position":0,"card_count":1},{"column_name":"tasks","date":"2020-12-14","position":0,"card_count":2},{"column_name":"tasks","date":"2020-12-14","position":0,"card_count":3},{"column_name":"tasks","date":"2020-12-14","position":0,"card_count":4},{"column_name":"tasks","date":"2020-12-14","position":0,"card_count":3},{"column_name":"tasks","date":"2020-12-14","position":0,"card_count":4},{"column_name":"tasks","date":"2020-12-14","position":0,"card_count":5},{"column_name":"tasks","date":"2020-12-14","position":0,"card_count":6},{"column_name":"tasks","date":"2021-01-25","position":0,"card_count":7},{"column_name":"tasks","date":"2021-02-09","position":0,"card_count":8},{"column_name":"tasks","date":"2021-02-15","position":0,"card_count":9},{"column_name":"tasks","date":"2021-02-15","position":0,"card_count":8},{"column_name":"tasks","date":"2021-02-22","position":0,"card_count":9},{"column_name":"tasks","date":"2021-02-26","position":0,"card_count":10},{"column_name":"tasks","date":"2021-02-26","position":0,"card_count":9},{"column_name":"tasks","date":"2021-03-08","position":0,"card_count":10},{"column_name":"im_gange","date":"2020-11-12","position":1,"card_count":0},{"column_name":"im_gange","date":"2020-11-30","position":1,"card_count":1},{"column_name":"im_gange","date":"2020-11-30","position":1,"card_count":2},{"column_name":"im_gange","date":"2020-11-30","position":1,"card_count":3},{"column_name":"im_gange","date":"2020-12-01","position":1,"card_count":4},{"column_name":"im_gange","date":"2020-12-01","position":1,"card_count":5},{"column_name":"im_gange","date":"2020-12-01","position":1,"

import de.tmrdlt.models._

import java.time.LocalDate

import scala.collection.mutable.ListBuffer


boards: Seq[de.tmrdlt.database.workflowlist.WorkflowList] = Vector(
  WorkflowList(
    1L,
    "5fad0555e165558a64ad0478",
    "Masterarbeit",
    Some(""),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-03-08T11:11:54.658
  )
)